In [2]:
from keras.datasets import cifar10
from keras import utils 
from keras.optimizers import Adam
import numpy as np 
from tensorflow.keras.models import load_model

In [3]:
dataset=cifar10
(x_train, y_train), (x_test, y_test) = dataset.load_data() #original size train(xxx, 28, 28),test(xxx)

#translate the data to image
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")

#normalization
x_train /= 255
x_test /= 255

lenet_cifar = load_model('Model/lenet5_cifar10.h5')  #load lenet5 model trained with mnist dataset 
lenet_cifar.compile(optimizer=Adam(),loss='sparse_categorical_crossentropy',metrics=['accuracy']) 

In [11]:
beta = 0.04057
beta= sample(x_test,y_test,beta)

Beta is reduced by 0.0001, the new beta is 0.04047
Beta is reduced by 0.0001, the new beta is 0.040369999999999996
Beta is reduced by 0.0001, the new beta is 0.04026999999999999
Beta is reduced by 0.0001, the new beta is 0.04016999999999999
Beta is reduced by 0.0001, the new beta is 0.04006999999999999
Beta is reduced by 0.0001, the new beta is 0.039969999999999985
Beta is reduced by 0.0001, the new beta is 0.03986999999999998
Beta is reduced by 0.0001, the new beta is 0.03976999999999998
Beta is reduced by 0.0001, the new beta is 0.039669999999999976
Beta is reduced by 0.0001, the new beta is 0.03956999999999997
Beta is reduced by 0.0001, the new beta is 0.03946999999999997
Beta is reduced by 0.0001, the new beta is 0.03936999999999997
Beta is reduced by 0.0001, the new beta is 0.039269999999999965
Beta is reduced by 0.0001, the new beta is 0.03916999999999996
Beta is reduced by 0.0001, the new beta is 0.03906999999999996
Beta is reduced by 0.0001, the new beta is 0.038969999999999956

0.025269999999999772

In [5]:
def sample(x_test,y_test,beta):
    
    mean = np.zeros(3)  
    cov = np.eye(3)
    
    for i in range(x_test.shape[0]):
        count = 0
        while True:
            p = np.random.multivariate_normal(mean, cov,(32,32)) #sample formula
            
            neighbour = p[0] + x_test[i]*255
            neighbour = neighbour.reshape(-1,32,32,3)
            neighbour /= 255
            distance=np.linalg.norm(p) 
            distance /= 255
            #print(distance)
            if (distance < beta or distance == beta):
                output = model.predict(neighbour,batch_size=128)  #prediction
                proba = softmax(output)
                pred = np.argmax(proba)
                label = y_test[i]
                if(pred == label): 
                    count+=1
                else:  #if there are some iamges inside beta cannot have the same label, beta will be decreased
                    beta = beta-0.0001
                    print("Beta is reduced by 0.0001, the new beta is",beta)  
                    i-=1
                    break
                    
            if(count<600):  
                continue
                
            if(count==600):  
                #local robustness formula satisfied
                break
                
    print("global robustness formula satisfied") 
    print("max safe distance:",beta)
        
    return beta

def softmax(x):
    e_x = np.exp(x-np.max(x))
    return e_x / e_x.sum()